In [3]:
# Cell 1 – Imports
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib


In [4]:
# Cell 2 – Load Dataset
df = pd.read_csv("../data/interim/StormEvents_cleaned.csv")

# If MONTH not present, derive it from MONTH_NAME
if "MONTH" not in df.columns:
    if "MONTH_NAME" in df.columns:
        month_map = {
            "JANUARY": 1, "FEBRUARY": 2, "MARCH": 3, "APRIL": 4,
            "MAY": 5, "JUNE": 6, "JULY": 7, "AUGUST": 8,
            "SEPTEMBER": 9, "OCTOBER": 10, "NOVEMBER": 11, "DECEMBER": 12
        }
        df["MONTH"] = df["MONTH_NAME"].astype(str).str.upper().map(month_map)



# Define features and target
FEATURES = ["EVENT_TYPE", "STATE", "MONTH",
            "MAGNITUDE", "MAGNITUDE_TYPE", "BEGIN_LAT", "BEGIN_LON"]

X = df[FEATURES]
y = df[["DAMAGE_PROPERTY", "DAMAGE_CROPS"]]

print("✅ Features shape:", X.shape)
print("✅ Targets shape:", y.shape)


✅ Features shape: (33904, 7)
✅ Targets shape: (33904, 2)


In [5]:
# Cell 3 – Define Preprocessor
categorical_features = ["EVENT_TYPE", "STATE",  "MAGNITUDE_TYPE"]
numeric_features = ["MONTH", "MAGNITUDE", "BEGIN_LAT", "BEGIN_LON"]

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=0, keep_empty_features=True)),
    ("scaler", StandardScaler())
])

preprocessor = ColumnTransformer([
    ("cat", categorical_transformer, categorical_features),
    ("num", numeric_transformer, numeric_features)
])


In [6]:
# Cell 4 – Define Models
models = {
    "LinearRegression": MultiOutputRegressor(LinearRegression()),
    "RandomForest": MultiOutputRegressor(RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)),
    "GradientBoosting": MultiOutputRegressor(GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, random_state=42)),
    "ExtraTrees": MultiOutputRegressor(ExtraTreesRegressor(n_estimators=300, random_state=42, n_jobs=-1))
}


In [7]:
# Cell 5 – Training & Evaluation
results = {}
best_model = None
best_score = -np.inf

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

for name, regressor in models.items():
    print(f"\n🔹 Training {name}...")
    
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("regressor", regressor)
    ])
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    metrics = {}
    r2_scores = []
    for i, target in enumerate(y.columns):
        rmse = np.sqrt(mean_squared_error(y_test.iloc[:, i], y_pred[:, i]))
        mae = mean_absolute_error(y_test.iloc[:, i], y_pred[:, i])
        r2 = r2_score(y_test.iloc[:, i], y_pred[:, i])
        metrics[target] = {"RMSE": rmse, "MAE": mae, "R2": r2}
        r2_scores.append(r2)
        print(f"{name} - {target}: RMSE={rmse:.2f}, MAE={mae:.2f}, R²={r2:.3f}")

    avg_r2 = np.mean(r2_scores)
    results[name] = {"metrics": metrics, "avg_r2": avg_r2}

    if avg_r2 > best_score:
        best_score = avg_r2
        best_model = (name, pipe)

print("\n✅ Best model selected:", best_model[0])



🔹 Training LinearRegression...
LinearRegression - DAMAGE_PROPERTY: RMSE=63934.48, MAE=14892.15, R²=0.960
LinearRegression - DAMAGE_CROPS: RMSE=29892.93, MAE=10030.53, R²=0.915

🔹 Training RandomForest...
RandomForest - DAMAGE_PROPERTY: RMSE=71362.32, MAE=19052.51, R²=0.950
RandomForest - DAMAGE_CROPS: RMSE=33327.41, MAE=10179.29, R²=0.894

🔹 Training GradientBoosting...
GradientBoosting - DAMAGE_PROPERTY: RMSE=64823.30, MAE=15880.16, R²=0.959
GradientBoosting - DAMAGE_CROPS: RMSE=30137.56, MAE=10074.91, R²=0.913

🔹 Training ExtraTrees...
ExtraTrees - DAMAGE_PROPERTY: RMSE=74676.94, MAE=19367.51, R²=0.945
ExtraTrees - DAMAGE_CROPS: RMSE=36045.29, MAE=10317.54, R²=0.876

✅ Best model selected: LinearRegression


In [8]:
# Cell 6 – Save Final Model
final_name, final_pipeline = best_model

joblib.dump(final_pipeline, "../models/storm_damage_model.joblib")

print(f"🌟 Final pipeline saved as storm_damage_model.joblib (Best: {final_name}, Avg R² = {best_score:.3f})")


🌟 Final pipeline saved as storm_damage_model.joblib (Best: LinearRegression, Avg R² = 0.937)


In [9]:
# Cell 7 – Compare Results
results_df = pd.DataFrame({
    (model, target): results[model]["metrics"][target]
    for model in results
    for target in results[model]["metrics"]
}).T

print("\n📊 Model Comparison:")
display(results_df)



📊 Model Comparison:


RMSE           MAE        R2
LinearRegression DAMAGE_PROPERTY  63934.479786  14892.151531  0.960003
                 DAMAGE_CROPS     29892.930891  10030.529499  0.914820
RandomForest     DAMAGE_PROPERTY  71362.322146  19052.514330  0.950169
                 DAMAGE_CROPS     33327.414443  10179.293664  0.894123
GradientBoosting DAMAGE_PROPERTY  64823.299531  15880.159620  0.958883
                 DAMAGE_CROPS     30137.563274  10074.905158  0.913420
ExtraTrees       DAMAGE_PROPERTY  74676.942794  19367.512261  0.945433
                 DAMAGE_CROPS     36045.287569  10317.538469  0.876150